In [1]:
!pip install datasets
!pip install gensim==4.3.1
!pip install tqdm
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from datasets import load_dataset
import pandas as pd
import json
import os
import re
from google.colab import drive
from tqdm import tqdm
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import nltk
import torch
import math
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from scipy.sparse import hstack
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from google.colab import drive
import os
drive.mount('/content/drive')
os.chdir("/content/drive/MyDrive/AI")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
hc3_df = pd.read_json(path_or_buf="eli5_clean_combined.jsonl", lines=True)
hc3_df

,question,human_answers,chatgpt_answers,index,scraped_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,NaN
...,...,...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,NaN,NaN,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,NaN,NaN,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,NaN,NaN,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,NaN,NaN,"[International waters, also known as the high ..."


In [5]:
# shift the data in scraped_answers to chatgpt_answers
for index, row in hc3_df.iterrows():
    # check if chatgpt_answers is nan
    if pd.isnull(row["chatgpt_answers"]).all() if type(row["chatgpt_answers"]) == list else pd.isnull(row["chatgpt_answers"]):
      hc3_df.at[index,"chatgpt_answers"] = row['scraped_answers']

hc3_df

,question,human_answers,chatgpt_answers,index,scraped_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,NaN
...,...,...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,[Closed-loop verification is a process used in...,NaN,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,[Mustard oil is not illegal to consume in the ...,NaN,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,[Space probes shoot pictures back at Earth usi...,NaN,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,"[International waters, also known as the high ...",NaN,"[International waters, also known as the high ..."


In [6]:
# drop index col which is all NaN
processed_hc3_df = hc3_df.drop(["scraped_answers"], axis=1)
processed_hc3_df = processed_hc3_df.drop(["index"], axis=1)
processed_hc3_df.dropna(inplace=True)
processed_hc3_df

,question,human_answers,chatgpt_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...
...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,"[International waters, also known as the high ..."


# Clean Dataset

In [7]:
# keep rows with non empty chatgpt ans
has_chatgpt_ans = processed_hc3_df["chatgpt_answers"].map(lambda ans_lst:ans_lst!=[])
processed_hc3_df = processed_hc3_df[has_chatgpt_ans]

#Need to expand the answers and chatgpt ans -- as for one question, there can be several human answers obtained
row_list = []
for index, row in processed_hc3_df.iterrows():
  human_ans_list = row["human_answers"]
  chatgpt_ans_list = row["chatgpt_answers"]
  # note that sometimes there can be 2 human ans, and 1 chatgpt ans for e.g. 
  num_chatgpt_ans = len(chatgpt_ans_list)
  for i in range(len(human_ans_list)):
    human_ans = human_ans_list[i]
    if i >= num_chatgpt_ans:
      chatgpt_ans = chatgpt_ans_list[num_chatgpt_ans-1]
    else:
      chatgpt_ans = chatgpt_ans_list[i]
    row_list.append([row["question"],human_ans,chatgpt_ans])

processed_hc3_df = pd.DataFrame(row_list, columns = ["question", "human_answer", "chatgpt_answer"])
print(processed_hc3_df.shape)
processed_hc3_df.head()

(68701, 3)


,question,human_answer,chatgpt_answer
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",There are many different best seller lists tha...
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",There are many different best seller lists tha...
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",There are many different best seller lists tha...
3,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,Salt is used on roads to help melt ice and sno...
4,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",Salt is used on roads to help melt ice and sno...


# Convert the shape of the df to n x 2, where the first column is the response and the second column is the label.

In [8]:
chatgpt_responses = processed_hc3_df.loc[:, ["chatgpt_answer"]]
chatgpt_responses["label"]=1
chatgpt_responses.rename(columns = {'chatgpt_answer':'response'}, inplace = True)
chatgpt_responses.drop_duplicates(inplace=True)
print(chatgpt_responses.shape)
chatgpt_responses.head()

(33366, 2)


,response,label
0,There are many different best seller lists tha...,1
3,Salt is used on roads to help melt ice and sno...,1
6,There are a few reasons why we still have SD (...,1
9,It is generally not acceptable or ethical to a...,1
12,After the Wright Brothers made the first power...,1


In [9]:
human_responses = processed_hc3_df.loc[:, ["human_answer"]]
human_responses["label"]=0
human_responses.rename(columns = {'human_answer':'response'}, inplace = True)
human_responses.drop_duplicates(inplace=True)
print(human_responses.shape)
human_responses.head()

(64250, 2)


,response,label
0,"Basically there are many categories of "" Best ...",0
1,"If you 're hearing about it , it 's because it...",0
2,"One reason is lots of catagories . However , h...",0
3,salt is good for not dying in car crashes and ...,0
4,"In Minnesota and North Dakota , they tend to u...",0


In [10]:
# sample the same number of human responses as chatgpt responses to make type equal in number
sampled_human_responses = human_responses.sample(n = chatgpt_responses.shape[0])
sampled_human_responses

,response,label
8891,Relevant : [ The hungarian suicide song ] ( UR...,0
22250,"Instead of imagining a 10 foot wave , imagine ...",0
49156,I was so confused because I thought it might b...,0
31866,Depends on the jump height . With a high enoug...,0
40436,I 've actually tested this with my two microwa...,0
...,...,...
2102,Funny how people is so enthusiastic about bad ...,0
66343,There are different perspectives and competing...,0
39933,Bone Marrow is mainly there for producing both...,0
66032,"No, you can't hear light if that's what you ar...",0


In [11]:
Corpus = pd.concat([chatgpt_responses, sampled_human_responses], ignore_index=True)
print(Corpus.shape)
Corpus.head()

(66732, 2)


,response,label
0,There are many different best seller lists tha...,1
1,Salt is used on roads to help melt ice and sno...,1
2,There are a few reasons why we still have SD (...,1
3,It is generally not acceptable or ethical to a...,1
4,After the Wright Brothers made the first power...,1


In [12]:
Corpus.dropna()
Corpus

,response,label
0,There are many different best seller lists tha...,1
1,Salt is used on roads to help melt ice and sno...,1
2,There are a few reasons why we still have SD (...,1
3,It is generally not acceptable or ethical to a...,1
4,After the Wright Brothers made the first power...,1
...,...,...
66727,Funny how people is so enthusiastic about bad ...,0
66728,There are different perspectives and competing...,0
66729,Bone Marrow is mainly there for producing both...,0
66730,"No, you can't hear light if that's what you ar...",0


# Save the transformed dataset

In [13]:
Corpus.to_csv("transformed_dataset.csv", index=False)